In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/4.2_hg38_paired_introns.tsv", sep="\t")
df

,chr,start,end,strand,class,ss_antisense_start_site,ss_antisense_end_site,GTExv2,TCGAv2,SRAv3h,...,repeat_features_end_site:Dust,repeat_features_end_site:Unknown,repeat_features_end_site:RNA repeats,repeat_features_end_site:Satellite repeats,repeat_features_end_site:Tandem repeats,repeat_features_end_site:Low complexity regions,sequence,sequence_maxentscan,hg38 - Acceptor splice site,hg38 - Donor splice site
0,chr1,12227,12612,+,1,False,False,1122:3199,583:1460,9374:54492,...,0,0,0,0,0,0,GTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGCCGG...,ccaGTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGC...,GT,AG
1,chr1,12721,13220,+,1,False,False,1791:3198,783:1104,14048:56719,...,0,0,0,0,0,0,GTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTAGGG...,gagGTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTA...,GT,AG
2,chr1,12057,12178,+,1,False,False,NaN,NaN,NaN,...,0,0,0,0,0,0,CACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAGTGG...,gagCACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAG...,CA,GA
3,chr1,12697,12974,+,1,False,False,1:1,1:1,15:19,...,0,0,0,0,0,0,GTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAGTGA...,cttGTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAG...,GT,AG
4,chr1,13052,13220,+,1,False,False,22:24,17:18,433:484,...,0,0,0,0,0,0,GCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGGAGA...,tagGCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGG...,GC,AG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527913,chrY,24883840,24886132,+,0,False,False,NaN,NaN,NaN,...,0,0,0,0,0,0,GTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAGGCCTT...,aatGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAGGC...,GT,GC
527914,chrY,24888605,24889352,+,0,False,False,NaN,NaN,NaN,...,0,0,0,0,0,0,GTAATGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAG...,cagGTAATGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATT...,GT,AG
527915,chrY,24889386,24901111,+,0,False,False,NaN,NaN,NaN,...,0,0,0,0,0,0,ATTATGTTTTCCTTGATGTTAAGTGAATTAGCCAAACATAGACTTC...,gggATTATGTTTTCCTTGATGTTAAGTGAATTAGCCAAACATAGAC...,AT,AG
527916,chrY,24833970,24840730,+,0,False,False,NaN,NaN,NaN,...,0,0,0,0,0,0,CCTTGGTTTTCCTTACACCTTAGCCTTTGGCTCCTTTGACCACTCG...,cgcCCTTGGTTTTCCTTACACCTTAGCCTTTGGCTCCTTTGACCAC...,CC,AG


In [3]:
# First we set all prediction to 1 (True/Accepted)
df["prediction"] = 1

using an minimum intron length threshold of 50 nt

In [4]:
df["intron_length"] = df["sequence"].apply(len)
df.loc[df["intron_length"] < 50, "prediction"] = 0

using a very limited definition of canonical splicing

In [5]:
#Acceptor and donor got mislabelled but not affecting the result
df.loc[(df["hg38 - Acceptor splice site"] == "GT") & df["hg38 - Donor splice site"] == "AG"] = 0
df.loc[(df["hg38 - Acceptor splice site"] == "GC") & df["hg38 - Donor splice site"] == "AG"] = 0
df.loc[(df["hg38 - Acceptor splice site"] == "AT") & df["hg38 - Donor splice site"] == "AC"] = 0

Recount3 Score not 0

In [6]:
df.loc[df["RC3-Score"] <= 0, "prediction"] = 0

using overlaps of splice sites with repeat elements - Transposable elements and tandem repeats

In [7]:
df.loc[df["repeat_features_end_site:Tandem repeats"] != 0, "prediction"] = 0
df.loc[df["repeat_features_start_site:Tandem repeats"] != 0, "prediction"] = 0
df.loc[df["repeat_features_start_site:Type I Transposons/SINE"] != 0, "prediction"] = 0
df.loc[df["repeat_features_end_site:Type I Transposons/SINE"] != 0, "prediction"] = 0
df.loc[df["repeat_features_start_site:Type I Transposons/LINE"] != 0, "prediction"] = 0
df.loc[df["repeat_features_end_site:Type I Transposons/LINE"] != 0, "prediction"] = 0

using overlap on the opposite strand with annotated transcripts

In [8]:
df.loc[df["ss_antisense_end_site"] == True, "prediction"] = 0
df.loc[df["ss_antisense_start_site"] == True, "prediction"] = 0

False positive under this filter:

In [11]:
df[(df["class"] != df["prediction"]) & (df.prediction == 1)]

,chr,start,end,strand,class,ss_antisense_start_site,ss_antisense_end_site,GTExv2,TCGAv2,SRAv3h,...,repeat_features_end_site:RNA repeats,repeat_features_end_site:Satellite repeats,repeat_features_end_site:Tandem repeats,repeat_features_end_site:Low complexity regions,sequence,sequence_maxentscan,hg38 - Acceptor splice site,hg38 - Donor splice site,prediction,intron_length
402492,chr10,120598210,120598626,+,0,False,False,0:0,0:0,1:17,...,0,0,0,0,GTGGTGTAGCTTTGCTGTCTCCTGTGTCAATGTCAGGAGGACAGGC...,tctGTGGTGTAGCTTTGCTGTCTCCTGTGTCAATGTCAGGAGGACA...,GT,AG,1,416
402493,chr10,130525340,130525712,+,0,False,False,4:4,1:1,5:15,...,0,0,0,0,GTAAGTGAAGGCTCGGGCTCCGGGAGGAGGTGCCGCCTTAGCATTG...,aagGTAAGTGAAGGCTCGGGCTCCGGGAGGAGGTGCCGCCTTAGCA...,GT,AG,1,372
402498,chr11,5249367,5254291,-,0,False,False,0:0,0:0,159:876,...,0,0,0,0,GTGAGTCCAGGAGATGTTTCAGCACTGTTGCCTTTAGTCTCGAGGC...,aagGTGAGTCCAGGAGATGTTTCAGCACTGTTGCCTTTAGTCTCGA...,GT,AG,1,4924
402523,chr15,21154963,21160305,+,0,False,False,1034:2932,269:417,5490:10573,...,0,0,0,0,GTAAGAACTGAGGGTGGCTGAGACTCGGGGTTGTTCAGGGCGGGGT...,aggGTAAGAACTGAGGGTGGCTGAGACTCGGGGTTGTTCAGGGCGG...,GT,AG,1,5342
402524,chr15,21165778,21167181,+,0,False,False,595:1807,862:1099,3219:6325,...,0,0,0,0,GTCTGTATATCTATATATTCTCATATTTATAAATCTCCATATTGTT...,aagGTCTGTATATCTATATATTCTCATATTTATAAATCTCCATATT...,GT,AG,1,1403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527827,chrY,9358328,9379213,+,0,False,False,150:245,61:322,102:359,...,0,0,0,0,GTATCCTTCTCAGTGTTTCTTCGGCCTTTCTAGTGGAGAGGTGCTC...,gttGTATCCTTCTCAGTGTTTCTTCGGCCTTTCTAGTGGAGAGGTG...,GT,AG,1,20885
527847,chrY,19739662,19741320,-,0,False,False,119:136,19:20,614:829,...,0,0,0,0,GTGGTGAGGGCCCTTAAACTGGTTTGTGGCCTGATGTGATTTTGTT...,catGTGGTGAGGGCCCTTAAACTGGTTTGTGGCCTGATGTGATTTT...,GT,AG,1,1658
527863,chrY,19735744,19739527,-,0,False,False,1:1,0:0,18:23,...,0,0,0,0,GTGAGTGACTGTTACTCCTGTTATTCTTCCTATAGCTGAGAAGGTT...,gatGTGAGTGACTGTTACTCCTGTTATTCTTCCTATAGCTGAGAAG...,GT,AG,1,3783
527882,chrY,23168943,23169385,-,0,False,False,0:0,0:0,36:60,...,0,0,0,0,GTAAGTAATCTAATAGAAAAATCTCTTATTTATCTTATTGCTACAA...,ggaGTAAGTAATCTAATAGAAAAATCTCTTATTTATCTTATTGCTA...,GT,AG,1,442


## 28407 False positive out of 125464 false introns